In [5]:
# immport dependencies
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pymongo
from pymongo import MongoClient
import geocoder

In [6]:
# MongoDB connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [7]:
# creates repeater db if it doesn't already exist
# drops stations table to avoid duplicate entries
db = client.repeater_db
db.stations.drop()

In [8]:
# url to scrape & soup setup
url = 'https://www.repeaterbook.com/repeaters/Display_SS.php?state_id=06&band=4&loc=%&call=%&use=%'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [9]:
# initialize variable to concat to location for use w/geocoder
california = ', CA'

In [10]:
# scrapes url & pushes data to stations table in db
table_rows = soup.find_all('tr')[4:14]
for item in table_rows:
    freq = item.find('a').text
    call_sign = item.find_all('td', attrs={'class': None})[3].text
    county = item.find_all('td', attrs={'class': None})[2].text
    location = item.find(class_="w3-left-align").text
    usage = item.find('font').text.strip()
    lat = geocoder.osm(location + california).lat
    lng = geocoder.osm(location + california).lng
    db.stations.insert_one(
        {'location': location,
         'latitude': lat,
         'longitude': lng,
         'frequency': freq,
         'call sign': call_sign,
         'county': county,
         'usage': usage})
                    
    print(location, lat, lng, freq, call_sign, county, usage)

Moreno Valley 33.937517 -117.2305944 425.4250 KC7NP Riverside OPEN
San Jose 37.3361905 -121.8905833 430.3000 W6IOS Santa Clara OPEN
Chico 39.7284945 -121.8374777 440.0000 KI6PNB Butte OPEN
Fremont 37.5482697 -121.9885719 440.0000 WA6FSP Alameda OPEN
Fresno 36.7295295 -119.708861260756 440.0025 N6VRC Fresno OPEN
Sonoma 38.5110803 -122.8473388 440.0125 W6PE Sonoma OPEN
Fremont 37.5482697 -121.9885719 440.0250 KI6KGN Alameda OPEN
Pacifica 37.6138253 -122.4869194 440.0250 N6TZE San Mateo OPEN
Ukiah 39.1501662 -123.2077861 440.0250 WA6RQZ Mendocino OPEN
Vacaville 38.3565773 -121.9877444 440.0250 WV6F Solano OPEN


In [15]:
# pushes mongo db table to pandas dataframe
stations = db.stations
df = pd.DataFrame(list(stations.find()))
df = df[['call sign', 'location', 'frequency', 'county', 'usage']]
df.head(10)

,call sign,location,frequency,county,usage
0,KC7NP,Moreno Valley,425.4250,Riverside,OPEN
1,W6IOS,San Jose,430.3000,Santa Clara,OPEN
2,KI6PNB,Chico,440.0000,Butte,OPEN
3,WA6FSP,Fremont,440.0000,Alameda,OPEN
4,N6VRC,Fresno,440.0025,Fresno,OPEN
5,W6PE,Sonoma,440.0125,Sonoma,OPEN
6,KI6KGN,Fremont,440.0250,Alameda,OPEN
7,N6TZE,Pacifica,440.0250,San Mateo,OPEN
8,WA6RQZ,Ukiah,440.0250,Mendocino,OPEN
9,WV6F,Vacaville,440.0250,Solano,OPEN
